<a href="https://colab.research.google.com/github/KaiaX926/P-Stocks-US/blob/main/SANA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import json
from urllib.request import urlopen
import datetime
import certifi
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import time

%matplotlib inline

YOUR_API_KEY = '2c2cd38706ef0b12bdbf80a33806aa32'
today = datetime.datetime.now().strftime("%Y-%m-%d")
startdate = (datetime.datetime.today() - datetime.timedelta(days = 366*3+31)).strftime("%Y-%m-%d")

- **fed funds rate** is the rate at which the fed suggests commercial banks borrow and lend their excess reserves to each other overnight. When the Fed raises the fed funds rate, it’s aiming to increase short-term interest rates throughout the economy. This in turn reduces the supply of money and makes it more expensive for all kinds of economic participants to borrow money. Lowering the fed funds rate has the opposite effect. It reduces short-term interest rates throughout the economy, increasing the supply of money and making it cheaper to get credit. This may cause moments of low or negative inflation to turn around and may drive hiring as companies are able to grow more cheaply.


- The **Consumer Price Index (CPI)** is a measure of the average change over time in the prices paid by urban consumers for a market basket of consumer goods and services. Indexes are available for the U.S. and various geographic areas. Average price data for select utility, automotive fuel, and food items are also available.

- **Nonfarm payrolls** is the measure of the number of workers in the U.S. excluding farm workers and workers in a handful of other job classifications. 

- **The Industrial Production Index (INDPRO)** is an economic indicator that measures real output for all facilities located in the United States manufacturing, mining, and electric, and gas utilities

In [2]:
class StockAnalysis():
    def __init__(self):
        self.YOUR_API_KEY = '2c2cd38706ef0b12bdbf80a33806aa32'
        self.today = datetime.datetime.now().strftime("%Y-%m-%d")
        self.startdate = (datetime.datetime.today() - datetime.timedelta(days = 366*3+31)).strftime("%Y-%m-%d")


    #####################################################################
    ## methods
    def get_data(self,url):
        response = urlopen(url)
        data = response.read().decode("utf-8")
        data = json.loads(data)
        return pd.json_normalize(data)

    def read_respond(self,ind,pe,da,response):
        data = response.read().decode("utf-8")
        data = data.replace('\n','')
        data = data.replace('{','')
        data = data.replace('}','')
        data = data[1:-1].split(',')
        #print(data)
        for i in data:
            if '"date" : "' in i:
                i = i.split(':')[1][2:-1]
                da.append(i)
            if '"industry" : "' in i:
                i = i.split(':')[1][2:-1]
                ind.append(i)
            if 'pe" : "' in i:
                p = round(float(i.split(':')[1][2:-2]),3)
                pe.append(p)
        return ind,pe,da

    #####################################################################
    # Analysis

    def get_eco(self):
        indicators = ['CPI','realGDP','durableGoods','totalNonfarmPayroll','industrialProductionTotalIndex','consumerSentiment']
        # cpi = 1, realGDP = /100, durableGoods = /1000, totalNonfarmPayroll = /1000, industrialProductionTotalIndex = 1, consumerSentiment = 1 (higher the more positive)
        indicators_rate = ['federalFunds','unemploymentRate','inflationRate']
        # federalFunds = 1,unemploymentRate = /10,inflationRate  = 1
        
        daily = pd.DataFrame(columns = ['indicator','date','value'])
        daily_rate = pd.DataFrame(columns = ['indicator','date','value'])
            
        for rate in indicators_rate:
            url_daily = f"https://financialmodelingprep.com/api/v4/economic?name={rate}&from={self.startdate}&to={self.today}&apikey={self.YOUR_API_KEY}"
            r = self.get_data(url_daily)
            if rate in ['unemploymentRate']:
                r['value'] = r['value']/10
            r['indicator'] = rate
            daily_rate = pd.concat([daily_rate,r], ignore_index = True)    
        fig_rate = px.line(daily_rate, x="date", y="value", color='indicator')
        fig_rate.show()
        
            
        for indica in indicators:
            url_daily = f"https://financialmodelingprep.com/api/v4/economic?name={indica}&from={self.startdate}&to={self.today}&apikey={self.YOUR_API_KEY}"
            d = self.get_data(url_daily)
            if indica in ['realGDP']:
                d['value'] = d['value']/100
            if indica in ['durableGoods','totalNonfarmPayroll']:
                d['value'] = d['value']/1000
            d['indicator'] = indica
            daily = pd.concat([daily,d], ignore_index = True)
        fig = px.line(daily, x="date", y="value", color='indicator')
        fig.show()
        
        return daily, daily_rate


    def get_overall(self):
        industry_list_NYSE = pd.DataFrame(columns = ['date','industry','pe'])
        industry_list_NASDAQ = pd.DataFrame(columns = ['date','industry','pe'])

        ind_NYSE,pe_NYSE,da_NYSE = [],[],[]
        ind_NASDAQ,pe_NASDAQ,da_NASDAQ = [],[],[]
        
        for day in range(1,365):
            date = (datetime.datetime.today() - datetime.timedelta(days=day)).strftime("%Y-%m-%d")
        
            #industry_list
            url_daily_industry_NYSE = f"https://financialmodelingprep.com/api/v4/industry_price_earning_ratio?date={date}&exchange=NYSE&apikey={self.YOUR_API_KEY}"
            url_daily_industry_NASDAQ = f"https://financialmodelingprep.com/api/v4/industry_price_earning_ratio?date={date}&exchange=NASDAQ&apikey={self.YOUR_API_KEY}"
            
            response_NYSE = urlopen(url_daily_industry_NYSE)
            response_NASDAQ = urlopen(url_daily_industry_NASDAQ)

            ind_NYSE,pe_NYSE,da_NYSE = self.read_respond(ind_NYSE,pe_NYSE,da_NYSE,response_NYSE)
            ind_NASDAQ,pe_NASDAQ,da_NASDAQ = self.read_respond(ind_NASDAQ,pe_NASDAQ,da_NASDAQ,response_NASDAQ)

        industry_list_NYSE['date'] = da_NYSE
        industry_list_NYSE['industry'] = ind_NYSE
        industry_list_NYSE['pe'] = pe_NYSE
        fig_NYSE = px.line(industry_list_NYSE, x="date", y="pe", color='industry',title='NYSE')
        fig_NYSE.show()
        print('NYSE:', np.unique(industry_list_NYSE['industry']))

        industry_list_NASDAQ['date'] = da_NASDAQ
        industry_list_NASDAQ['industry'] = ind_NASDAQ
        industry_list_NASDAQ['pe'] = pe_NASDAQ
        fig_NASDAQ = px.line(industry_list_NASDAQ, x="date", y="pe", color='industry',title='NASDAQ')
        fig_NASDAQ.show()
        print('NASDAQ:', np.unique(industry_list_NASDAQ['industry']))

        return industry_list_NYSE, industry_list_NASDAQ


    def find_industry(self,ind):
        
        url1 = f'https://financialmodelingprep.com/api/v3/quotes/nyse?apikey={self.YOUR_API_KEY}'
        url2 = f'https://financialmodelingprep.com/api/v3/quotes/nasdaq?apikey={self.YOUR_API_KEY}'
        stocks1 = self.get_data(url1)#.columns
        stocks2 = self.get_data(url2)#.columns

        stocks = np.unique(list(stocks1['symbol']) + list(stocks2['symbol']))
        print('Total stocks considered: ',len(stocks))
        data_profile = pd.DataFrame(columns=['symbol','industry', 'price', 'range', 'beta','isEtf', 'isFund'])
        data_score = pd.DataFrame(columns = ['symbol','altmanZScore','piotroskiScore'])
        data_income_statement = pd.DataFrame(columns = ['symbol','reportedCurrency','grossProfitRatio','ebitdaratio','operatingIncomeRatio','netIncomeRatio'])
        data_suggestion = pd.DataFrame(columns=['symbol', 'date', 'rating', 'ratingScore','ratingDetailsDCFScore','ratingDetailsROEScore', 
                                  'ratingDetailsROAScore', 'ratingDetailsDEScore','ratingDetailsPEScore', 'ratingDetailsPBScore'])

        url_sic = f'https://financialmodelingprep.com/api/v4/standard_industrial_classification_list?apikey={YOUR_API_KEY}'
        data_sic = self.get_data(url_sic)
        indslist = list(data_sic['industryTitle'])
        siccode = []
        for i in range(len(indslist)):
            for dd in ind:
                if dd.lower() in indslist[i].lower():
                    siccode.append(data_sic['sicCode'][i])
        print(siccode)        
        url_stocks = f'https://financialmodelingprep.com/api/v4/standard_industrial_classification/all?apikey={YOUR_API_KEY}'
        data_stocks = self.get_data(url_stocks)
        companies_or = list(data_stocks[data_stocks['sicCode'].isin(siccode)]['symbol'])
        print(companies_or)

        for sym in companies_or:
            #print('yes')
            url_profile = f"https://financialmodelingprep.com/api/v3/profile/{sym}?apikey={self.YOUR_API_KEY}"
            x = self.get_data(url_profile)
            if len(x) > 0:
                #print(x['symbol'][0],x['isin'][0])
                if x['currency'][0] in ['USD','AUD','EUR','HKD','CNY'] and x['isin'][0]:
                    x = x[['symbol','industry', 'price', 'range', 'beta','isEtf', 'isFund']]
                    data_profile = pd.concat([data_profile,x], ignore_index = True)
            # else:
            #     companies.remove(sym)

        #####################################################################
        # Personal preference
        data_profile = data_profile[data_profile.price > 20] 

        companies = list(data_profile['symbol'])
        print('companies list is ready')

        data_profile = data_profile.set_index('symbol')
        pricerange = data_profile['range'].str.split('-', expand=True)
        data_profile['currentLoc'] = (data_profile['price'] - pricerange[0].astype('float'))/(pricerange[1].astype('float') - pricerange[0].astype('float')).astype('float64')
        data_profile.currentLoc = data_profile.currentLoc.astype('float')
        data_profile = data_profile.round(2)


        for sym in companies:
            url_score = f"https://financialmodelingprep.com/api/v4/score?symbol={sym}&apikey={self.YOUR_API_KEY}"
            url_income_statement = f"https://financialmodelingprep.com/api/v3/income-statement/{sym}?apikey={self.YOUR_API_KEY}"
            url_suggestion = f"https://financialmodelingprep.com/api/v3/rating/{sym}?apikey={self.YOUR_API_KEY}"
            
            score = self.get_data(url_score)
            if len(score) == 0:
                score = pd.DataFrame([[sym,'','']], columns = ['symbol','altmanZScore','piotroskiScore'])
            #score['symbol'] = sym
            
            income_statement = self.get_data(url_income_statement)
            if len(income_statement) == 0:
                income_statement = pd.DataFrame([[sym,'','','','','']], columns = ['symbol','reportedCurrency','grossProfitRatio','ebitdaratio','operatingIncomeRatio','netIncomeRatio'])
            else:
                income_statement = pd.DataFrame(income_statement.iloc[0,:]).T

            suggestion = self.get_data(url_suggestion)
            if len(suggestion) == 0:
                suggestion = pd.DataFrame([[sym,'','','','','','','','','',]], 
                        columns=['symbol', 'date', 'rating', 'ratingScore','ratingDetailsDCFScore','ratingDetailsROEScore', 
                                  'ratingDetailsROAScore', 'ratingDetailsDEScore','ratingDetailsPEScore', 'ratingDetailsPBScore'])
            else:               
                suggestion = suggestion[['symbol', 'date', 'rating', 'ratingScore','ratingDetailsDCFScore','ratingDetailsROEScore', 
                                  'ratingDetailsROAScore', 'ratingDetailsDEScore','ratingDetailsPEScore', 'ratingDetailsPBScore']]


            data_score = pd.concat([data_score,score[['symbol','altmanZScore','piotroskiScore']]], ignore_index = True)
            data_income_statement = pd.concat([data_income_statement, income_statement[['symbol','reportedCurrency','grossProfitRatio','ebitdaratio','operatingIncomeRatio','netIncomeRatio']]],ignore_index = True)
            data_suggestion = pd.concat([data_suggestion, suggestion],ignore_index = True)
      
        print('companies three elements are ready')
        data_score = data_score.set_index('symbol')
        data_income_statement = data_income_statement.set_index('symbol')
        data_suggestion = data_suggestion.set_index('symbol')
        
        data_profile = data_profile.join(data_score, how = 'left')
        data_profile = data_profile.join(data_income_statement, how = 'left')
        
        url_sentiment = f"https://financialmodelingprep.com/api/v4/social-sentiment/trending?apikey={self.YOUR_API_KEY}"
        data_sentiment = self.get_data(url_sentiment)       
        data_sentiment =  data_sentiment[['symbol','stocktwitsSentiment','twitterSentiment']]
        data_sentiment = data_sentiment.set_index('symbol')
        data_profile = data_profile.join(data_sentiment, how = 'left')
        
        url_sentiment_change = f"https://financialmodelingprep.com/api/v4/social-sentiments/change?type=bullish&source=stocktwits&apikey={self.YOUR_API_KEY}"
        data_sentiment_change = self.get_data(url_sentiment_change)       
        data_sentiment_change =  data_sentiment_change[['symbol','sentimentChange','rank']]
        data_sentiment_change.rename(columns={'rank': 'sentimentChangeRank'}, inplace=True)
        data_sentiment_change = data_sentiment_change.set_index('symbol')
        data_profile = data_profile.join(data_sentiment_change, how = 'left')
                
        return data_profile, data_suggestion
            
    def stock(self,compines):
        data_stock = pd.DataFrame(columns = ["symbol","date","open","high","low","close","volume","wma"])
        data_sentiment = pd.DataFrame(columns = ['symbol','date','stocktwitsSentiment','twitterSentiment'])
        
        for sym in compines:
            url_price = f"https://financialmodelingprep.com/api/v3/technical_indicator/daily/{sym}?period=10&type=wma&apikey={self.YOUR_API_KEY}"
            url_sentiment = f"https://financialmodelingprep.com/api/v4/historical/social-sentiment?symbol={sym}&page=0&apikey={self.YOUR_API_KEY}"

            
            stock = self.get_data(url_price)
            stock['symbol'] = sym
            data_stock = pd.concat([data_stock,stock], ignore_index = True)

            fig = go.Figure(data=[go.Candlestick(
                x=stock['date'],
                open=stock['open'],
                high=stock['high'],
                low=stock['low'],
                close=stock['close'])],
                layout_title_text=sym
                )
            fig.update_layout(xaxis_rangeslider_visible=False)
            fig.show()
        
            sentiment = self.get_data(url_sentiment)
            if len(sentiment) > 0:
                sentiment = sentiment[['symbol','date','stocktwitsSentiment','twitterSentiment']]
                data_sentiment = pd.concat([data_sentiment,sentiment], ignore_index = True)
        
        if len(data_sentiment) > 0:
            fig_sentiment = px.line(data_sentiment, x="date", y="stocktwitsSentiment", color='symbol')
            fig_sentiment.show()
        
        return data_stock,data_sentiment

        
    def mockowen(self,compines, datebuy, datesell, invest):
        data_stock = pd.DataFrame(columns = ["symbol","open","high","low","close","wma",'flucRate','earning'])
        
        for sym in compines:
            url_price = f"https://financialmodelingprep.com/api/v3/technical_indicator/daily/{sym}?period=10&type=wma&apikey={self.YOUR_API_KEY}"

            stock = self.get_data(url_price)
            stock = stock.set_index("date")
            stock['symbol'] = sym
            stock['flucRate'] = (stock.open - stock.open.shift(-1))/stock.open.shift(-1)
            stock['earning'] = stock.open - stock.open[datebuy]
            data_stock = pd.concat([data_stock,stock.loc[datesell:datebuy, :]])
            print(f"income on stock {sym} is: {invest/stock['open'][datebuy] * (stock['open'][datesell] - stock['open'][datebuy]):0.4f}")
        
            fig_stock = px.line(stock.loc[datesell:datebuy, :], y=["earning",'flucRate'])
            fig_stock.show()
        return data_stock



In [3]:

if __name__ == "__main__":
    start_time = time.time()
    SANA = StockAnalysis()
    daily, daily_rate = SANA.get_eco()
    industry_list_NYSE, industry_list_NASDAQ = SANA.get_overall()

 #   inds = input("Please enter your Industry preference(exp: [A, B, C]): ").strip('][').split(', ')
    inds = ['house', 'drug']
    print(inds)
    stocks_profile, stocks_suggestion = SANA.find_industry(inds)
    print("Total time consuming --- %s seconds ---" % (time.time() - start_time))

NYSE: ['Airlines' 'Auto Manufacturers' 'Banks Diversified' 'Banks—Diversified'
 'Building Materials' 'Closed-End Fund - Equity'
 'Closed-End Fund - Foreign' 'Computer Hardware' 'Consulting Services'
 'Drug Manufacturers General' 'Drug Manufacturers—General'
 'Footwear & Accessories' 'Household & Personal Products'
 'Internet Content & Information' 'Metals & Mining'
 'Oil & Gas Equipment & Services' 'Oil & Gas Midstream'
 'Other Industrial Metals & Mining' 'REIT Diversified' 'REIT—Diversified'
 'Telecom Services' 'Trucking' 'Utilities Diversified'
 'Utilities Independent Power Producers' 'Utilities—Regulated Electric']


NASDAQ: ['Confectioners' 'Credit Services' 'Drug Manufacturers General'
 'Drug Manufacturers—General' 'Entertainment' 'Financial Conglomerates'
 'Insurance—Reinsurance' 'Internet Retail' 'Leisure' 'Oil & Gas E&P'
 'Packaged Foods' 'REIT Mortgage' 'REIT—Mortgage' 'Railroads'
 'Real Estate Services' 'Semiconductors' 'Steel' 'Thermal Coal'
 'Utilities Regulated Electric' 'Utilities—Regulated Electric']
['house', 'drug']
Total stocks considered:  9647
['2510', '2511', '3630', '3634', '3651', '5122', '5912', '5961', '7000']
['ABC', 'ABWN', 'ALID', 'AMZN', 'AOS', 'APRN', 'ARLO', 'AXIN', 'BSET', 'BWMX', 'BZUN', 'CAH', 'CANB', 'CDW', 'CHWY', 'CJJD', 'CNXN', 'CODI', 'CODI-PA', 'CODI-PB', 'CODI-PC', 'COSM', 'CPNG', 'CSPR', 'CVEO', 'CVET', 'CVS', 'DGJI', 'EMGE', 'ETH', 'EWLU', 'FLXS', 'GNSS', 'HBB', 'HELE', 'HEWA', 'HLF', 'HNST', 'HOFT', 'ICTV', 'IMBI', 'IMTE', 'INSD', 'IPSI', 'IRBT', 'JMIA', 'KBAL', 'KMGH', 'KN', 'KOSS', 'LEG', 'LEPX', 'LITB', 'LUVU', 'LZB', 'MCK', 'MEDS', 'MOHO', 'MSN', 'MWK', 

In [4]:
stocks_profile


,industry,price,range,beta,isEtf,isFund,currentLoc,altmanZScore,piotroskiScore,reportedCurrency,grossProfitRatio,ebitdaratio,operatingIncomeRatio,netIncomeRatio,stocktwitsSentiment,twitterSentiment,sentimentChange,sentimentChangeRank
symbol,,,,,,,,,,,,,,,,,,
ABC,Medical Distribution,155.14,111.34-156.25,0.46,False,False,0.98,4.326553,5.0,USD,0.032447,0.01362,0.011001,0.007196,NaN,NaN,NaN,NaN
AMZN,Internet Retail,3326.02,2671.45-3773.08,1.11,False,False,0.59,5.365535,6.0,USD,0.260454,0.15806,0.052954,0.071014,0.4990,0.6213,NaN,NaN
AOS,Specialty Industrial Machinery,66.25,60.39-86.74,1.19,False,False,0.22,6.794206,6.0,USD,0.370426,0.200006,0.0,0.137642,NaN,NaN,NaN,NaN
CAH,Medical Distribution,57.63,45.85-62.11,0.91,False,False,0.72,4.268797,7.0,USD,0.041719,0.007909,0.002905,0.003761,NaN,NaN,NaN,NaN
CDW,Information Technology Services,178.82,162.47-208.71,1.12,False,False,0.35,3.226567,5.0,USD,0.171391,0.069579,0.068153,0.047481,NaN,NaN,NaN,NaN
CHWY,Internet Retail,42.79,35.59-97.74,0.51,False,False,0.12,7.919398,2.0,USD,0.266971,-0.001931,-0.008118,-0.008303,0.5000,0.0000,NaN,NaN
CNXN,Electronics & Computer Distribution,53.64,40.08-54.79,0.64,False,False,0.92,6.48073,4.0,USD,0.16061,0.037587,0.033367,0.024167,NaN,NaN,NaN,NaN
CODI,Conglomerates,24.31,21.84-33.31,1.82,False,False,0.22,1.50328,6.0,USD,0.394185,0.167169,0.075748,0.0622,NaN,NaN,NaN,NaN
CODI-PA,Conglomerates,24.79,23.14-27.38,1.82,False,False,0.39,1.350883,6.0,USD,0.394185,0.167169,0.075748,0.0622,NaN,NaN,NaN,NaN


In [5]:
stocks_suggestion

,date,rating,ratingScore,ratingDetailsDCFScore,ratingDetailsROEScore,ratingDetailsROAScore,ratingDetailsDEScore,ratingDetailsPEScore,ratingDetailsPBScore
symbol,,,,,,,,,
ABC,2022-03-29,S,5,5,5,3,5,5,5
AMZN,2022-03-29,S-,5,5,4,3,5,5,5
AOS,2022-03-29,A+,5,5,3,3,4,5,5
CAH,2022-03-29,S-,5,5,3,3,5,5,5
CDW,2022-03-29,S,5,5,5,3,5,5,5
CHWY,2022-03-29,B+,4,5,1,3,5,1,5
CNXN,2022-03-29,A+,5,5,3,3,4,5,5
CODI,2022-03-29,A+,5,5,3,3,4,5,5
CODI-PA,2022-03-29,A+,5,5,3,3,4,5,5


In [ ]:
if __name__ == "__main__":
    SANA = StockAnalysis()
    stocks = input("Please enter Stock symbols on your watchlist (exp: [A,B,C]): ").strip('][').split(', ')
    data_stock,data_sentiment = SANA.stock(stocks)

    wl_stocks = input("Please enter Stock symbols on your watchlist (exp: [A,B,C]): ").strip('][').split(', ')
    wl_dates = input("Please enter dates to buy and sellS (exp: [2021-02-25,2022-02-25]): ").strip('][').split(', ')
    wl_money = float(input("Please enter invest money per stock(exp: 100): "))
    mock_stock= SANA.mockowen(wl_stocks, wl_dates[0], wl_dates[1], wl_money)